In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
#load dataset from packages
mnist_ds, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [3]:
#Pre-processing
#split train and test sample data
mnist_train, mnist_test = mnist_ds['train'], mnist_ds['test']

#store train samples
n_valid_samples = 0.1 * mnist_info.splits['train'].num_examples
n_valid_samples = tf.cast(n_valid_samples, tf.int64) #cast variable to an int

#store test samples
n_test_samples = 0.1 * mnist_info.splits['test'].num_examples
n_test_samples = tf.cast(n_test_samples, tf.int64) #cast variable to an int

#scaling data
def scale(image, label):
    image = tf.cast(image, tf.float32) #cast image to float
    image /= 255.
    return image, label

scaled_train_valid_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000 #batches to be shuffled at intervals

shuffled_train_valid_data = scaled_train_valid_data.shuffle(BUFFER_SIZE)

valid_data = shuffled_train_valid_data.take(n_valid_samples)
train_data = shuffled_train_valid_data.skip(n_valid_samples)

BATCH_SIZE = 100 #batch size for mini batch GD

train_data= train_data.batch(BATCH_SIZE)
valid_data = valid_data.batch(n_valid_samples) #overwrite with whole size of valid data
test_data = test_data.batch(n_test_samples)

valid_inputs, valid_targets = next(iter(valid_data))

In [4]:
#Outlining the model
#set hyperparameters

input_size = 784
output_size = 10
hiddenlayer_size = 100

#Specifying Layers
model = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hiddenlayer_size,activation='relu'), #first hidden layer
                             tf.keras.layers.Dense(hiddenlayer_size,activation='relu'), #second hidden layer
                             tf.keras.layers.Dense(output_size,activation='softmax') #output layer
])

In [5]:
#Choose optimizer and loss fucntion
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
#Model training
N_EPOCHS = 5

model.fit(train_data, epochs= N_EPOCHS, validation_data=(valid_inputs, valid_targets), verbose=2)


Epoch 1/5
540/540 - 3s - loss: 0.3419 - accuracy: 0.9029 - val_loss: 0.1796 - val_accuracy: 0.9465
Epoch 2/5
540/540 - 2s - loss: 0.1420 - accuracy: 0.9577 - val_loss: 0.1222 - val_accuracy: 0.9627
Epoch 3/5
540/540 - 3s - loss: 0.0985 - accuracy: 0.9705 - val_loss: 0.0972 - val_accuracy: 0.9703
Epoch 4/5
540/540 - 3s - loss: 0.0735 - accuracy: 0.9780 - val_loss: 0.0817 - val_accuracy: 0.9757
Epoch 5/5
540/540 - 3s - loss: 0.0593 - accuracy: 0.9811 - val_loss: 0.0627 - val_accuracy: 0.9798


In [7]:
#Testing the model
test_loss, test_accuracy = model.evaluate(test_data)

10/10 [==============================] - 1s 69ms/step - loss: 0.0868 - accuracy: 0.9732


In [8]:
print('Test Loss: {0:.2f} // Test Accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

Test Loss: 0.09 // Test Accuracy: 97.32%
